In [6]:
import datetime
# import plotly packages 
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder
le=LabelEncoder()

plt.style.use("seaborn-whitegrid")
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm_notebook
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ss = pd.read_csv('submission_sample.csv')
train['ind'] = 1
test['ind'] = 0
df= pd.concat([train, test])
obj=df.select_dtypes('object')
num=df.select_dtypes(exclude='object')
df

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,ind
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92.0,11.000000,60.200001,...,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118,1
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91.0,14.600000,48.799999,...,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652,1
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96.0,16.400000,33.400002,...,-78.342701,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96.0,6.911948,21.300001,...,-73.896588,30.545446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95.0,13.900001,44.700001,...,-68.612480,26.899694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16131,ZZDJZMS X 2020-03-31,2020-03-31,ZZDJZMS,NaN,NaN,NaN,NaN,NaN,9.400001,86.200005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16132,ZZDJZMS X 2020-04-01,2020-04-01,ZZDJZMS,NaN,NaN,NaN,NaN,NaN,17.000000,90.300003,...,45.983723,40.416741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16133,ZZDJZMS X 2020-04-02,2020-04-02,ZZDJZMS,NaN,NaN,NaN,NaN,NaN,19.170641,86.200005,...,-152.281488,41.202583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16134,ZZDJZMS X 2020-04-03,2020-04-03,ZZDJZMS,NaN,NaN,NaN,NaN,NaN,8.400001,66.400002,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [7]:
col=[ 'target_min','target_max', 'target_variance', 'target_count',]
df.drop(col,axis=1,inplace=True) 

drop_col=[col for col in df.columns if 'L3_CH4_' in col ]
df.drop(columns=drop_col,axis=1,inplace=True)
  
   
   
df['month'] = pd.to_datetime(df['Date']).dt.month
df['weekofyear'] = pd.to_datetime(df['Date']).dt.weekofyear
df['day'] = pd.to_datetime(df['Date']).dt.day

df['sin_day'] = np.sin(2 * np.pi * df['day'] / 30)
df['cos_day'] = np.cos(2 * np.pi * df['day'] / 30)
df['sin_weekofyear'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
df['cos_weekofyear'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
for col in num:
    for i in tqdm_notebook(range(1, 3)):

        df[f'prev_{i}_{col}'] = df.sort_values(by='Date').groupby('Place_ID')[col].shift(i).sort_index()
        df[f'dif_pr_{i}_{col}'] = df[col] - df[f'prev_{i}_{col}']
        df[f'next_{i}_{col}'] = df.sort_values(by='Date').groupby('Place_ID')[col].shift(-i).sort_index()
        df[f'dif_next_{i}_{col}'] = df[col] -df[f'next_{i}_{col}']

  0%|          | 0/2 [00:00<?, ?it/s]

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
train = df[df.ind == 1]
test = df[df.ind == 0]
from sklearn.model_selection  import train_test_split
X=train.drop(['Place_ID X Date','Date','target'],axis=1)
y=train.target
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
model = CatBoostRegressor().fit(X,y)
xgr = XGBRegressor().fit(X,y)
lgr = LGBMRegressor().fit(X,y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scaling or normalize or log

In [27]:
# from sklearn.preprocessing import StandardScaler,scale

# X=scale(X)
# sc_test=scale(test[main_col])

In [28]:
from sklearn.metrics import mean_squared_error
import math

y_predicted = model.predict(X_test)
 
MSE = mean_squared_error(y_test, (y_predicted))
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Model Accuracy: 0.605


In [5]:
pred_df= pd.DataFrame({'Actual': y_test, 'Predicted': model.predict(X_test)})  
pred_df

NameError: name 'y_test' is not defined

In [4]:
px.line(xgr_df[:5])

NameError: name 'xgr_df' is not defined

In [29]:

from sklearn.metrics import mean_squared_error, r2_score
y_train_pred = lgr.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

# Predict on the testing set and calculate mean squared error (MSE) and R-squared value
y_test_pred = lgr.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Training set RMSE: {:.3f}, R-squared: {:.3f}".format(rmse_train, r2_train))
print("Testing set RMSE: {:.3f}, R-squared: {:.3f}".format(rmse_test, r2_test))

Training set RMSE: 24.218, R-squared: 0.736
Testing set RMSE: 29.132, R-squared: 0.605


In [1]:
from sklearn.model_selection import cross_validate
lgr = LGBMRegressor().fit(X_train, y_train)
y= pd.qcut(y, q=10, labels=False)
def evaluate(model, X, y, cv):
    cv_results = cross_validate(model,X, y,cv=cv,scoring=['neg_mean_absolute_error',"r2", "neg_root_mean_squared_error"],)
  
    mae = -cv_results["test_neg_mean_absolute_error"]
    r2 = cv_results["test_r2"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"r2 or accuracy:     {r2.mean():.3f}\n"
        f"Mean Absolute Error:     {mae.mean():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} ")
    return 


evaluate(lgr, X, y, cv=5)

NameError: name 'LGBMRegressor' is not defined

In [11]:
pred=lgr.predict(sc_test)
new
sub = pd.DataFrame()
sub['Place_ID X Date'] = test['Place_ID X Date']
sub['target'] = pred
sub.to_csv(".csv", index=False)

NameError: name 'sc_test' is not defined